In [1]:
import json
kmb_loaded = json.load(open('./bus.json', encoding='utf-8-sig'))
kmb_loaded = kmb_loaded['features']
print("Total number of bus stops: ", len(kmb_loaded))

Total number of bus stops:  54051


In [2]:
from langchain_text_splitters import RecursiveJsonSplitter

splitter = RecursiveJsonSplitter()
splits = splitter.create_documents(kmb_loaded)
print(splits[0:5])
print("Total number of splits: ", len(splits))

[Document(metadata={}, page_content='{"type": "Feature", "geometry": {"type": "Point", "coordinates": [114.192446, 22.345353]}, "properties": {"routeId": 1001, "companyCode": "KMB", "district": "", "routeNameC": "1", "routeNameS": "1", "routeNameE": "1", "routeType": 1, "serviceMode": "R", "specialType": 0, "journeyTime": 49, "locStartNameC": "\\u7af9\\u5712\\u90a8", "locStartNameS": "\\u7af9\\u56ed\\u90a8", "locStartNameE": "CHUK YUEN ESTATE", "locEndNameC": "\\u5c16\\u6c99\\u5480\\u78bc\\u982d", "locEndNameS": "\\u5c16\\u6c99\\u5480\\u7801\\u5934", "locEndNameE": "STAR FERRY", "hyperlinkC": "https://search.kmb.hk/KMBWebSite/?action=routesearch&route=1&lang=zh-hk", "hyperlinkS": "https://search.kmb.hk/KMBWebSite/?action=routesearch&route=1&lang=zh-cn", "hyperlinkE": "https://search.kmb.hk/KMBWebSite/?action=routesearch&route=1&lang=en", "fullFare": 6.4, "lastUpdateDate": "2023-07-07T00:00:00", "routeSeq": 1, "stopSeq": 1, "stopId": 4001, "stopPickDrop": 2, "stopNameC": "\\u7af9\\u5712

In [3]:
def divide_chunks(l, n):
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]

chunks = list(divide_chunks(splits, 500))
print("Total number of chunks: ", len(chunks))

Total number of chunks:  109


: 

In [4]:
from google.oauth2.service_account import Credentials
from langchain_chroma import Chroma
from langchain_google_vertexai import VertexAIEmbeddings

credentials = Credentials.from_service_account_file(
    "ive-fyp-436703-3a208c6d96a0.json")

vertexAIEmbeddings = VertexAIEmbeddings(
    credentials=credentials,
    project=credentials.project_id,
    model_name="text-multilingual-embedding-002",
)

vertex_store = Chroma(
    collection_name='KMB',
    embedding_function=vertexAIEmbeddings,
    persist_directory='./chroma_db',
)

for c, i in enumerate(chunks):
    print(vertex_store.add_documents(i))
    print(f"Added {c+1} chunks")